In [1]:
from models.byol_parallel import BYOL_parallel
from models.byol_pytorch import BYOL
from data.custom_transforms import BatchTransform, ListToTensor, PadToSquare, SelectFromTuple
from data.pairs_dataset import PairsDataset, pair_collate_fn

from data.multi_pairs_dataset import MultiPairsDataset
import numpy as np
import os

import sys
import torch
import torch.nn as nn

import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torchvision import datasets
import warnings


# distributed training
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler


In [2]:
epochs = 3

In [5]:
class TransFormSquare:

    def __init__(self, size):

        self.size = size
        self.x_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                PadToSquare(255),
                T.Resize((self.size,self.size)),
            ]
        )

        self.y_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                PadToSquare(255),
                T.Resize((self.size,self.size)),
            ]
        )

    def __call__(self, x, y):
        return self.x_transform(x) , self.y_transform(y)


In [6]:
ecommerce = {"images" : '/home/vision/smb-datasets/ecommerce/images/train/',
             "sketches" : '/home/vision/smb-datasets/ecommerce/ecommerce_pidinet/'}

save_file = 'bimodal_byol_shoes_utils/checkpoints/Test/test_BYOl_parallel_training.pt'
pretrained_model = 'bimodal_byol_shoes_utils/checkpoints/bimodal_byol_resnet50_pretrained_sketchy_v5.pt'


ecommerce_dataset = PairsDataset(
    ecommerce["images"],
    ecommerce["sketches"],
    transform=TransFormSquare(224)
)



In [7]:

def setup(rank, worls_size):

    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"

    dist.init_process_group("nccl", rank=rank, world_size=worls_size)

def prepare(rank, world_soze, batch_size=32, pin_memory=False, num_workers=0):

    dataset = ecommerce_dataset
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank,
                                 shuffle=False, drop_last=False)

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=pin_memory,
        num_workers=num_workers,
        drop_last=False,
        shuffle=False,
        sampler=sampler)

def cleanup():
    dist.destroy_process_group()



In [8]:
encoder = models.resnet50(weights="IMAGENET1K_V2")

empty_transform = T.Compose([])

learner = BYOL(
    encoder,
    image_size=224,
    hidden_layer="avgpool",
    augment_fn=empty_transform
    #cosine_ema_steps=epochs*epoch_size
)

torch.save(learner.state_dict(), save_file.format(epochs))
learner.load_state_dict(torch.load( save_file.format(epochs)))

# optimizador
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

hooking
cpu
dict_keys([device(type='cpu')])
cpu
dict_keys([device(type='cpu')])
cpu
dict_keys([device(type='cpu')])
cpu
dict_keys([device(type='cpu')])


In [9]:
def main(rank, world_size):
    #setup the process groups
    setup(rank, world_size)

    #prepare the dataloader
    dataloader = prepare(rank, world_size)

    #instaDP

    # wrap the model with DDP
    # device_ids tell DDP where is your model
    # output_device tells DDP where to output, in our case, it is rank
    # find_unused_parameters=True instructs DDP to find unused output of the forward() function of any module in the model

    model = DDP(learner, device_ids=[rank], output_device=rank, find_unused_parameters=True)

    optimizer = opt()

    running_loss = np.array([], dtype=np.float32)
    for epoch in range(epochs):
        # if we are using DistributedSampler, we have to tell it whicj epoch this is
        dataloader.sampler.set_epoch(epoch)

        for step, (x,y) in enumerate(dataloader):
            optimizer.zero_grad(set_to_none=True)

            loss = learner(x,y)#.to('cuda', dtype=torch.float)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            learner.module.update_moving_average()

            running_loss = np.append(running_loss, [loss.mean().item()])
            sys.stdout.write('\rEpoch {}, batch {} - loss {:.4f}'.format(epoch+1, i+1, np.mean(running_loss)))

            #i += 1
            #if i%(epoch_size/2)==0:
            #    torch.save(learner.state_dict(), save_file.format(epoch + 1))


        torch.save(learner.state_dict(), save_file.format(epoch + 1))
        running_loss = np.array([], dtype=np.float32)
        sys.stdout.write('\n')

    cleanup()


In [10]:

world_size = 2

mp.spawn(main,
     args=(world_size),
     nprocs=world_size)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'main' on <module '__main__' (built-in)>


ProcessExitedException: process 1 terminated with exit code 1

In [ ]:
encoder = models.resnet50(weights="IMAGENET1K_V2")

empty_transform = T.Compose([])

learner = BYOL(
    encoder,
    image_size=224,
    hidden_layer="avgpool",
    augment_fn=empty_transform
    #cosine_ema_steps=epochs*epoch_size
)

learner = DDP(learner)


# optimizador
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

epochs = 3
epoch_size = len(train_loader)

hooking
cpu
dict_keys([device(type='cpu')])
cpu
dict_keys([device(type='cpu')])


In [ ]:
torch.save(learner.state_dict(), save_file.format(epochs))
learner.load_state_dict(torch.load( save_file.format(epochs)))

learner.to(device)

In [8]:

learner.train()
filehandler = open('bimodal_byol_shoes_utils/training_bimodal_byol.txt', 'w')
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    running_loss = np.array([], dtype=np.float32)
    for epoch in range(epochs):
        i = 0

        #for images in Loaders[int(epoch/epochsForLoader)]:
        for step, (x,y) in enumerate(train_loader):
            print(step)

            loss = learner(x,y)#.to('cuda', dtype=torch.float)
            opt.zero_grad()
            loss.mean().backward()
            opt.step()
            learner.module.update_moving_average()

            running_loss = np.append(running_loss, [loss.mean().item()])
            sys.stdout.write('\rEpoch {}, batch {} - loss {:.4f}'.format(epoch+1, i+1, np.mean(running_loss)))
            filehandler.write('Epoch {}, batch {} - loss {:.4f}\n'.format(epoch+1, i+1, np.mean(running_loss)))
            filehandler.flush()
            i += 1
            if i%(epoch_size/2)==0:
                torch.save(learner.state_dict(), save_file.format(epoch + 1))


        torch.save(learner.state_dict(), save_file.format(epoch + 1))
        running_loss = np.array([], dtype=np.float32)
        sys.stdout.write('\n')
        break
filehandler.close()

0
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:1
dict_keys([device(type='cuda', index=1)])
cuda:1
dict_keys([device(type='cuda', index=1)])
cuda:1
dict_keys([device(type='cuda', index=1)])
cuda:1
dict_keys([device(type='cuda', index=1)])
Epoch 1, batch 1 - loss 3.92631
cuda:0
dict_keys([device(type='cuda', index=0), device(type='cuda', index=1)])
cuda:1
dict_keys([])
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:0
dict_keys([device(type='cuda', index=0)])
cuda:0
dict_keys([device(type='cuda', index=0)])


KeyError: Caught KeyError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/users/btorres/proyectos/redesNeuronales/venv/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/users/btorres/proyectos/redesNeuronales/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/users/btorres/proyectos/redesNeuronales/models/byol_parallel.py", line 223, in forward
    online_proj_one = self.online_encoder(image_one)
  File "/users/btorres/proyectos/redesNeuronales/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/users/btorres/proyectos/redesNeuronales/models/byol_parallel.py", line 170, in forward
    representation = self.get_representation(x)
  File "/users/btorres/proyectos/redesNeuronales/models/byol_parallel.py", li